```shell
Created: 28 Apr 2018
Last modified: 28 Apr 2018
```

# Notes on `mongo`
> **Contents**
1. `mongo` basic
2. `mongo` shell

## 1. `mongo` basics

### 1.1 `mongo` data structure –– BSON

**MongoDB** uses **JSON**(JavaScript Object Notation) doucuments in binary-encoded format, which provides many different types and more complicated structure. **BSON** or Binary JSON, extends the JSON model to provide additional data types. 

**Example of JSON**
```JSON
{"menu": {
  "id": "file",
  "value": "File",
  "popup": {
    "menuitem": [
      {"value": "New", "onclick": "CreateNewDoc()"},
      {"value": "Open", "onclick": "OpenDoc()"},
      {"value": "Close", "onclick": "CloseDoc()"}
    ]
  }
}
```
**Compare with BSON**
```BSON
{"_id":123,
 "name":{
     "first":"first name",
     "last":"last name"
 },
 "age":29,
 "Belongings":[
     {"item":"shoes", "value":200},
     {"item":"bicycle", "value":800},
     {"item":"briefcase", "value":1500}
 ]
}
```

## 1.2 Basic CRUD in `mongo`

### 1.2.1 Creation
To create a new database, a new collection, a new document
* `use <database>`: switch to another database. If it doesn't exist, it will initialise a new database.
* `db.createCollection(<collection>)`: initialise a new collection.
* Using variable to temporarily store a document:
```mongo
> var new = {"sid":123,
...            "name":"David",
...            "info":{"year":2018, "mark":90}
...            }
> db.collection.insert(new)
```

### 1.2.2 Read
* `find()`: return the result of a query. Without given parameter, it will by default return the entire documents in a collection.  
```mongo
> db.collection.find()
```
* `count()`: return the number of documents.
* `sort()`: sort the documents by criteria. `1` for ascending, `-1` for descending.
```mongo
> db.collection.find().sort({"_id":1})
{"_id":123, ...}
{"_id":124, ...}
...
```
* `limit()`: limit the documents to retrieved. No limit if parameter is not given.
* `hasNext()` and `next()`:  
Considering the following codes:
```shell
> var results = db.collection.find({"unit_code":"FIT5148"})
> results.hasNext() #Check whether it is empty.
true
> results.count()
2
> results.next()
{"sid":1, ...}
> results.next()
{"sid":2, ...}
> results.hasNext()
false
> results.count()
2
> results.next()
2018-04-28T22:42:53.058+1000 E QUERY    [thread1] Error: error hasNext: false :
DBQuery.prototype.next@src/mongo/shell/query.js:305:1
@(shell):1:1
```
Now we can see that the `results` variable stores the output of the `find()`. It works as a generator in Python  but "pop" the first document out when execute `next()` function and remember where it ends. It is noteworthy to notice that `count()` only retrieve the number of documents in this variable(which seem to be immutable after the variable has been initialised).  
* `print()`: print out the document.
* `pretty()` or `printjson()`: print out in a more readable style.
* `explain()`: it usually take `"allPlansExecution"` as its parameter. More infomation in **Indexing**.

### 1.2.3 Update  
To update document(s) of a collection
* `update()`:
* `updateOne()`:
* `updateMany()`:
* `replaceOne()`:
* Another way to populate data: Run JavaScript in terminal

### 1.2.4 Deletion
* `db.<collection>.remove()`: delete documents by query criteria.
* `db.<collection>.drop()`: delete the entire collection.
* `db.dropDatabase()`: drop all collections of the current database.

### 3. `mongo` shell 

Date - 28 Apr 2018

Run `JavaScript` in `mongo` shell
---
Let's take a `JavaScript` file as example:  
> `example.js`
```javascript
db = connect("localhost:27017/another") //Specify which database to run
db.evaluation.insertMany(...)
```

Then, we can run this `JavaScript` file in terminal with `mongo` shell:
```
$ mongo example.js
```
It works exactly the same as we are working in the `mongo` shell
```shell
> use another
> db.evaluation.insertMany(...)
```

---

Date - 29 Apr 2018

Aggregation: aggregate values & re-shaping documents
---
**Aggregation pipeline**
> 
```
{$match} ➜ {$project} ➜ {$group} ➜ {$sort}
 filter  ➜   reshape  ➜  sum-up  ➜  order     
```

Let's take this dataset as example:
```
> db.qoh.find({})
{ "_id" : 1, "type" : 2, "supplier" : "OFF", "qty" : 1000 }
{ "_id" : 2, "type" : 2, "supplier" : "OFF", "qty" : 500 }
{ "_id" : 3, "type" : 1, "supplier" : "ON", "qty" : 200 }
{ "_id" : 4, "type" : 3, "supplier" : "OFF", "qty" : 1000 }
{ "_id" : 5, "type" : 1, "supplier" : "ON", "qty" : 40 }
{ "_id" : 6, "type" : 2, "supplier" : "ON", "qty" : 1000 }
{ "_id" : 7, "type" : 3, "supplier" : "ON", "qty" : 5000 }
```

First, let's <font color=red>**group**</font> documents with ***same type***, then respectively <font color=red>**sum**</font> the quantity ***"qty"***. Noted that ***"total"*** is a newly created key temporarily for storing value of the sum of ***"qty"***. ( <font color=red>**Re-shaping**</font> )

```
> db.qoh.aggregate(
...                 [
...                  {$group:{_id: "$type", total: {$sum: "$qty"}}}
...                 ]
...                )
{ "_id" : 3, "total" : 6000 }
{ "_id" : 1, "total" : 240 }
{ "_id" : 2, "total" : 2500 }
```

How about extract document which "supplier" matches "ON". ( <font color=red>**Matching**</font> )
```
> db.qoh.aggregate(
...                 [
...                  {$match:{supplier: "ON"}}
...                 ]
...                )
{ "_id" : 3, "type" : 1, "supplier" : "ON", "qty" : 200 }
{ "_id" : 5, "type" : 1, "supplier" : "ON", "qty" : 40 }
{ "_id" : 6, "type" : 2, "supplier" : "ON", "qty" : 1000 }
{ "_id" : 7, "type" : 3, "supplier" : "ON", "qty" : 5000 }
```

Now, let's combine these command into one. We want to have document which ***"supplier"*** is ***"ON"***, then grouped by their ***"type"*** and sum the quantity ***"qty"***, and do sorting the ***"total"*** in ascending order.
```
> db.qoh.aggregate(
...                 [
...                  {$match: {supplier: "ON"}},
...                  {$group: {_id: "$type", total: {$sum: "$qty"}}},
...                  {$sort: {total:1}}
...                 ]
...                )
{ "_id" : 1, "total" : 240 }
{ "_id" : 2, "total" : 1000 }
{ "_id" : 3, "total" : 5000 }
```

---